In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
more_features = True
use_staking = False
use_kernal_params = True
do_augment = True
fast = False

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [5]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [6]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [7]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [8]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [9]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
predictions = test_df[['ID_code']]
val_aucs = []

In [11]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 5
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Fri Mar 22 22:30:09 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.887	valid_1's auc: 0.827718
[600]	training's auc: 0.896238	valid_1's auc: 0.838558
[900]	training's auc: 0.903961	valid_1's auc: 0.848416
[1200]	training's auc: 0.910296	valid_1's auc: 0.856002
[1500]	training's auc: 0.915522	valid_1's auc: 0.862158
[1800]	training's auc: 0.919365	valid_1's auc: 0.866252
[2100]	training's auc: 0.922728	valid_1's auc: 0.869765
[2400]	training's auc: 0.925644	valid_1's auc: 0.872874
[2700]	training's auc: 0.928273	valid_1's auc: 0.875298
[3000]	training's auc: 0.930445	valid_1's auc: 0.877177
[3300]	training's auc: 0.932311	valid_1's auc: 0.878673
[3600]	training's auc: 0.93416	valid_1's auc: 0.880005
[3900]	training's auc: 0.935698	valid_1's auc: 0.881071
[4200]	training's auc: 0.937067	valid_1's auc: 0.88212
[4500]	training's auc: 0.938329	valid_1's auc: 0.883015
[4800]	training's auc: 0.939501	valid_1's auc: 0.88361
[5100]

[23700]	training's auc: 0.973338	valid_1's auc: 0.888482
[24000]	training's auc: 0.973711	valid_1's auc: 0.888503
[24300]	training's auc: 0.974077	valid_1's auc: 0.888501
[24600]	training's auc: 0.974431	valid_1's auc: 0.888445
[24900]	training's auc: 0.974789	valid_1's auc: 0.888418
[25200]	training's auc: 0.97515	valid_1's auc: 0.888429
[25500]	training's auc: 0.975502	valid_1's auc: 0.888439
[25800]	training's auc: 0.975843	valid_1's auc: 0.888451
[26100]	training's auc: 0.976184	valid_1's auc: 0.888402
[26400]	training's auc: 0.976529	valid_1's auc: 0.888418
Early stopping, best iteration is:
[23555]	training's auc: 0.97316	valid_1's auc: 0.888522
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.886661	valid_1's auc: 0.8268
[600]	training's auc: 0.896324	valid_1's auc: 0.838066
[900]	training's auc: 0.904207	valid_1's auc: 0.848379
[1200]	training's auc: 0.910324	valid_1's auc: 0.85585
[1500]	training's auc: 0.91548	valid_1's auc: 0.862093
[18

[15300]	training's auc: 0.961403	valid_1's auc: 0.888201
[15600]	training's auc: 0.961881	valid_1's auc: 0.888223
[15900]	training's auc: 0.962366	valid_1's auc: 0.888242
[16200]	training's auc: 0.962827	valid_1's auc: 0.888303
[16500]	training's auc: 0.96331	valid_1's auc: 0.888296
[16800]	training's auc: 0.963764	valid_1's auc: 0.888281
[17100]	training's auc: 0.964221	valid_1's auc: 0.888281
[17400]	training's auc: 0.964669	valid_1's auc: 0.888282
[17700]	training's auc: 0.965121	valid_1's auc: 0.888322
[18000]	training's auc: 0.965561	valid_1's auc: 0.888303
[18300]	training's auc: 0.966013	valid_1's auc: 0.888311
[18600]	training's auc: 0.966452	valid_1's auc: 0.888316
[18900]	training's auc: 0.966875	valid_1's auc: 0.888279
[19200]	training's auc: 0.967316	valid_1's auc: 0.888307
[19500]	training's auc: 0.96774	valid_1's auc: 0.888332
[19800]	training's auc: 0.968166	valid_1's auc: 0.888391
[20100]	training's auc: 0.968598	valid_1's auc: 0.888423
[20400]	training's auc: 0.969015	

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 1 started at Fri Mar 22 23:32:17 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.88624	valid_1's auc: 0.830887
[600]	training's auc: 0.89614	valid_1's auc: 0.842138
[900]	training's auc: 0.904187	valid_1's auc: 0.850742
[1200]	training's auc: 0.910385	valid_1's auc: 0.857773
[1500]	training's auc: 0.915403	valid_1's auc: 0.863858
[1800]	training's auc: 0.919184	valid_1's auc: 0.868027
[2100]	training's auc: 0.922436	valid_1's auc: 0.871381
[2400]	training's auc: 0.925241	valid_1's auc: 0.873945
[2700]	training's auc: 0.927915	valid_1's auc: 0.876217
[3000]	training's auc: 0.930127	valid_1's auc: 0.878206
[3300]	training's auc: 0.932069	valid_1's auc: 0.879719
[3600]	training's auc: 0.933905	valid_1's auc: 0.880986
[3900]	training's auc: 0.935509	valid_1's auc: 0.882024
[4200]	training's auc: 0.936885	valid_1's auc: 0.882916
[4500]	training's auc: 0.938173	valid_1's auc: 0.883685
[4800]	training's auc: 0.93932	valid_1's auc: 0.884283
[51

...

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

Fold 4 started at Sat Mar 23 02:41:01 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.887332	valid_1's auc: 0.827511
[600]	training's auc: 0.896126	valid_1's auc: 0.835538
[900]	training's auc: 0.903842	valid_1's auc: 0.844912
[1200]	training's auc: 0.91006	valid_1's auc: 0.852505
[1500]	training's auc: 0.915259	valid_1's auc: 0.858619
[1800]	training's auc: 0.919163	valid_1's auc: 0.86323
[2100]	training's auc: 0.922394	valid_1's auc: 0.866863
[2400]	training's auc: 0.925256	valid_1's auc: 0.870069
[2700]	training's auc: 0.927791	valid_1's auc: 0.872653
[3000]	training's auc: 0.930033	valid_1's auc: 0.874823
[3300]	training's auc: 0.932006	valid_1's auc: 0.876492
[3600]	training's auc: 0.933852	valid_1's auc: 0.877963
[3900]	training's auc: 0.935422	valid_1's auc: 0.879259
[4200]	training's auc: 0.936806	valid_1's auc: 0.880253
[4500]	training's auc: 0.938073	valid_1's auc: 0.881149
[4800]	training's auc: 0.939233	valid_1's auc: 0.881796
[5

[1500]	training's auc: 0.915765	valid_1's auc: 0.858072
[1800]	training's auc: 0.919528	valid_1's auc: 0.862504
[2100]	training's auc: 0.922751	valid_1's auc: 0.866153
[2400]	training's auc: 0.925622	valid_1's auc: 0.869391
[2700]	training's auc: 0.928091	valid_1's auc: 0.871895
[3000]	training's auc: 0.930321	valid_1's auc: 0.874057
[3300]	training's auc: 0.932181	valid_1's auc: 0.875595
[3600]	training's auc: 0.933999	valid_1's auc: 0.877047
[3900]	training's auc: 0.935533	valid_1's auc: 0.878389
[4200]	training's auc: 0.936926	valid_1's auc: 0.879507
[4500]	training's auc: 0.938148	valid_1's auc: 0.880296
[4800]	training's auc: 0.93929	valid_1's auc: 0.88109
[5100]	training's auc: 0.940381	valid_1's auc: 0.881806
[5400]	training's auc: 0.941374	valid_1's auc: 0.882392
[5700]	training's auc: 0.942318	valid_1's auc: 0.88282
[6000]	training's auc: 0.943198	valid_1's auc: 0.883302
[6300]	training's auc: 0.944039	valid_1's auc: 0.88368
[6600]	training's auc: 0.944843	valid_1's auc: 0.884

[4800]	training's auc: 0.939353	valid_1's auc: 0.881596
[5100]	training's auc: 0.940429	valid_1's auc: 0.882203
[5400]	training's auc: 0.941436	valid_1's auc: 0.882706
[5700]	training's auc: 0.942392	valid_1's auc: 0.883154
[6000]	training's auc: 0.943278	valid_1's auc: 0.883631
[6300]	training's auc: 0.94411	valid_1's auc: 0.884007
[6600]	training's auc: 0.944903	valid_1's auc: 0.884338
[6900]	training's auc: 0.945655	valid_1's auc: 0.884561
[7200]	training's auc: 0.946361	valid_1's auc: 0.884795
[7500]	training's auc: 0.947066	valid_1's auc: 0.885037
[7800]	training's auc: 0.947745	valid_1's auc: 0.885243
[8100]	training's auc: 0.948423	valid_1's auc: 0.885366
[8400]	training's auc: 0.949042	valid_1's auc: 0.885545
[8700]	training's auc: 0.94968	valid_1's auc: 0.885697
[9000]	training's auc: 0.950291	valid_1's auc: 0.885785
[9300]	training's auc: 0.950881	valid_1's auc: 0.885884
[9600]	training's auc: 0.951471	valid_1's auc: 0.885996
[9900]	training's auc: 0.952042	valid_1's auc: 0.8

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [12]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.891135677, std: 0.003832281. All auc: 0.891087850.


In [13]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
